In [64]:
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
import quaternion
import re
import datetime
import os
import glob
from MOF_build.functions.rotate import rotate
from MOF_build.functions.read import read
from MOF_build.functions.output import output

In [65]:
'''translate and rotate node to beginning point'''

#node = read.pdb('PCN222_lib_nodeZr.pdb')
node = read.pdb('node_lib.pdb')
#print(node.iloc[30,:],node.iloc[57,:])
array = node.loc[:, ['x','y','z']].to_numpy()
center = 0.5*(array[47]+array[46]) #center point (beginning point)
new_node = array-center
#uio66
#any_tdx, any_tdy, any_tdz = (
#np.array([1, 0, 0]),
#np.array([0, 1, 0]),
#np.array([0, 1, 2]))

#PCN222
any_tdx, any_tdy, any_tdz = (
    np.array([1, 0, 0]),
    np.array([0, 1, 0]),
    np.array([0, 1, 1]))
dx, dy, dz = np.array([1, 0, 0]), np.array([0, 1, 0]), np.array([0, 0, 1])
carte_basis = np.vstack([dx, dy, dz])
  
tric_basis = rotate.coordinate_transfer(any_tdx, any_tdy, any_tdz, carte_basis)
#v1_file = new_node[9]-new_node[0]
#v2_file = new_node[27]-new_node[18]
v1_file = new_node[47]-new_node[46]
v2_file = new_node[40]-new_node[19]
v1_frame = tric_basis[2]+tric_basis[1]
v2_frame = tric_basis[0]
rotated_node = rotate.rotate_twice_linker(node,center,v1_file,v1_frame,v2_file,v2_frame)
#rotateq_x90=rotate.calculate_q_rotation_with_axis_degree(tric_basis[0],np.pi/2)
#rotated_node = rotate.get_rotated_array(rotated_node0,rotateq_x90)
node.loc[:, ['x','y','z']] = rotated_node
newdf = pd.concat([node.iloc[:,0:3],node.iloc[:,6],node.iloc[:,3:6]],axis=1, join = 'outer') #move ['NOTE'] to column3
newdf.to_csv('PCNZRRnodeOO.txt',header=None,sep='\t',index=False)
output('PCNZRRnodeOO',True,False,False)
print(newdf)

   Atom_label Residue  Res_number  Note             x         y         z
0          O2     MOL           2   O1- -9.634869e-01 -0.138695  3.941472
1          O1     MOL           2   O1- -1.787529e+00 -1.987784  2.999589
2          C2     MOL           2     C -1.757411e+00 -1.847868  5.354320
3          C1     MOL           2     C -1.496753e+00 -1.272895  3.977297
4          O2     MOL           2   O1-  1.621625e+00 -2.286254 -2.937025
5          O1     MOL           2   O1-  2.984144e+00 -2.421214 -1.173323
6          C2     MOL           2     C  3.320926e+00 -3.916093 -2.966268
7          C1     MOL           2     C  2.592677e+00 -2.765102 -2.304704
8          O2     MOL           2   O1-  1.622608e+00  0.460243  3.692962
9          O1     MOL           2   O1-  2.984026e+00 -0.882471  2.541609
10         C2     MOL           2     C  3.322618e+00 -0.671557  4.865628
11         C1     MOL           2     C  2.593355e+00 -0.325736  3.584002
12         O2     MOL           2   O1

In [66]:
#reset resnumber  
justnode = read.pdb('PCNZRRnodeOO.pdb') #ZRRnode.pdb
justnode.iloc[:,2]=1 #set resnumber as 1
#print(justnode)
df = pd.concat([justnode.iloc[:,0:3],justnode.iloc[:,6],justnode.iloc[:,3:6]],axis=1, join = 'outer')
df.to_csv('lib_ZRnode.txt',header=None,sep='\t',index=False)
output('lib_ZRnode',True,False,False)


In [67]:
'''add dummy atom to Zr node'''
pre_dummy_pdb = newdf

atom_type = []
for i in pre_dummy_pdb['Atom_label']:
    atom_type.append(re.sub(r"\d", "", i))

pre_dummy_pdb.insert(7, "atom_type", atom_type, True)

df_Zr =pre_dummy_pdb[pre_dummy_pdb['atom_type']=='Zr'].reset_index()
df_O =pre_dummy_pdb[pre_dummy_pdb['atom_type']=='O'].reset_index()
O_positions = df_O.loc[:, ['x','y','z']].to_numpy()

df_add_dummy=pd.DataFrame()
for i in range(df_Zr.shape[0]):
    df_dummy = read.pdb("dummy_template.pdb")
    distance_O = []
    distance_Osort = []
    pointZR = df_Zr.loc[i, ['x','y','z']].to_numpy()
    for j in range(len(O_positions)):
        distance_O.append(np.linalg.norm(pointZR-O_positions[j]) )
        distance_Osort.append(np.linalg.norm(pointZR-O_positions[j]) )
    distance_Osort.sort()
    threshhold = distance_Osort[8]
    indices = [index for index, num in enumerate(distance_O) if num < threshhold]
    #print(indices,distance_O)
    dummy_atoms = np.zeros((8,3))
    row =0
    for k in indices:
        newD = pointZR+(O_positions[k]-pointZR)/np.linalg.norm(pointZR-O_positions[i])
        dummy_atoms[row,:]=newD
        row +=1
    print(pointZR,'\n')
    dummy_atoms=np.round(dummy_atoms,3)
    df_dummy.loc[0, ['x','y','z']]=pointZR
    df_dummy.loc[1:, ['x', 'y', 'z']] = dummy_atoms
    df_dummy.loc[:,'Res_number'] = i+1
  
    df_add_dummy = pd.concat([df_add_dummy,df_dummy],ignore_index=True)

df_add_dummy



[-1.2828322361721944 -1.9734979015079257 0.8175743003429508] 

[2.1389472401115692 -1.180907045381434 0.48902300987540626] 

[-2.1389472401115674 1.1809070453814317 -0.48902300987540326] 

[1.2828322361721962 1.9734979015079235 -0.8175743003429479] 

[-1.7763568394002505e-15 -0.9408341451932061 -2.271374553269135] 

[-4.440892098500626e-16 0.9408341451932067 2.2713745532691383] 



,Atom_label,Residue,Res_number,x,y,z,Note
0,Zr,ZRR,1,-1.282832e+00,-1.973498,0.817574,Zr
1,D1,ZRR,1,-1.422000e+00,-1.977000,1.418000,D
2,D2,ZRR,1,-1.422000e+00,-2.401000,0.396000,D
3,D3,ZRR,1,-1.884000e+00,-2.064000,0.855000,D
4,D4,ZRR,1,-1.097000e+00,-2.506000,1.038000,D
5,D5,ZRR,1,-8.120000e-01,-1.740000,1.077000,D
6,D6,ZRR,1,-8.120000e-01,-1.992000,0.470000,D
7,D7,ZRR,1,-1.410000e+00,-1.671000,0.326000,D
8,D8,ZRR,1,-1.410000e+00,-1.413000,0.951000,D
9,Zr,ZRR,2,2.138947e+00,-1.180907,0.489023,Zr


In [68]:
'''output'''
df_add_dummy.insert(3, "note", df_add_dummy['Note'], True)
df_add_dummy.to_csv('0adddummy.txt',header=None,sep='\t',index=False)
#
output('0adddummy',True,False,False)